# Task 2: The Byte Pair Encoding Algorithm

---

## Part 1: Setting up the foundations

### The BPE Algorithm

The Byte Pair Encoding algorithm is an iterative approach to building a subword vocabulary. It starts by treating each character in the training corpus as an initial token. Then, it repeatedly finds the most frequent pair of adjacent tokens and merges them into a new single token. This process is repeated for a specified number of merges, which determines the final vocabulary size.

### Scenario 1: A Minimalist Implementation

Your task is to implement the core BPE algorithm for a small, controlled text corpus.

#### Problem Description

You are given the following training text:

`'a b c a b c a a b c a a'`

Your task is to implement a function that performs a single BPE merge on this text.  

#### Implementation Steps

1. Initialize the vocabulary. Each character is a token.
2. Count the frequency of all adjacent token pairs.
3. Identify the most frequent pair.
4. Merge the most frequent pair into a new token.
5. Return the new token list and the new vocabulary.

In [1]:
from collections import Counter

# 1.Spliting into tokens and initializing vocabulary
given_text = 'a b c a b c a a b c a a'
tokens = given_text.split() # this will be generating character level tokens
print(f"Initial Tokens: {tokens}")
vocab = set(tokens)
print(f"Initial Vocab: {vocab}")
pairs = []

# 2. Counting frequency of all adjacent token pairs
for i in range(len(tokens)-1):
    pair = (tokens[i], tokens[i+1])
    pairs.append(pair)
pair_counts = Counter(pairs)
print(f"Pair counts: {pair_counts}")

# 3.identifying the most frequent pair
most_frequent_pair = pair_counts.most_common(1)[0][0] 

print(f"Most Frequent Pair: {most_frequent_pair}")
print(" ------ After FIRST merge ------ ")

# 4. Merging the most frequent pair into a new token
merging_most_frequent_pair = "".join(most_frequent_pair)

# 5. New tokens
i = 0
new_tokens = []
while i < len(tokens):
    if i < len(tokens) - 1 and (tokens[i], tokens[i+1]) == most_frequent_pair:
        new_tokens.append(merging_most_frequent_pair)
        i += 2
    else:
        new_tokens.append(tokens[i])
        i +=1
print(f"New tokens: {new_tokens}")

# 6. Returning a new vocabulary
vocab.add(merging_most_frequent_pair)
new_vocab = vocab
print(f"New Vocab: {new_vocab}")

Initial Tokens: ['a', 'b', 'c', 'a', 'b', 'c', 'a', 'a', 'b', 'c', 'a', 'a']
Initial Vocab: {'c', 'a', 'b'}
Pair counts: Counter({('a', 'b'): 3, ('b', 'c'): 3, ('c', 'a'): 3, ('a', 'a'): 2})
Most Frequent Pair: ('a', 'b')
 ------ After FIRST merge ------ 
New tokens: ['ab', 'c', 'ab', 'c', 'a', 'ab', 'c', 'a', 'a']
New Vocab: {'c', 'ab', 'a', 'b'}


## Reflective Questions

**1. Why is it important to start with characters as the initial tokens?**  
Starting with characters as tokens provides a meaningful base vocabulary. It ensures that algorithm has seen those words which are not even there during training. Through adjacent merges individual character as tokens can be converted into syllables, words, sub words.  

**2. What is the purpose of counting adjacent pairs?**  
It helps in identifying the most frequent sequences occuring together. By merging the most common pairs, the algorithm gradually builds larger, more meaningful tokens that reduce sequence length and improve efficiency.  

**3. If there were a tie for the most frequent pair, how would you resolve it? What is a common strategy in BPE implementations?**. 
There are multiple ways of resolution in case of a tie for the most frequent pair e-g. picking the first occuring pair, or use lexicographic ordering. In case of my implementation i have used lexicographic ordering for picking up the most frequent pair in case of a tie.  


---

## Part 2: Extending the Implementation

### Scenario 2: Training a BPE Model

Now, you will build upon your initial implementation to create a full BPE training function. This function will take a text corpus and a specified number of merges as input and will generate a vocabulary and a tokenizer.  

#### Implementation Steps
1. Implement the `train` function to obtain the tokens for the given text corpus. The function should return the final vocabulary of the tokenizer after it has been trained.
2. Iteratively perform `num_merges` merges. 
In each iteration:  
    a. Count pair frequencies.  
    b. Identify and store the most frequent pair and its new token.  
    c. Merge the pair throughout the text corpus.  
3. Implement the `tokenize` function that takes a new string as input and tokenizes it using the learned merges and vocabulary. This function must handle new words by breaking them down into known subwords or characters.  

#### Problem Statement
You have to perform the following tasks:

- Train the tokenizer on the following text:

    `'the dog is a good boy, the cat is a good girl'`

    - Print the vocabulary of the tokenizer
    - Use num_merges = 5

- Tokenize the following sentence and output the result:

    `'the good dog is a boy'`

In [ ]:
class BPE:
    def __init__(self):
        self.vocab = set()
        self.merges = []

    def train(self, text_corpus, num_merges):
        # 1. Initializing tokens and vocabulary (character level)
        tokens = list(text_corpus)  # keeping spaces
        self.vocab = set(tokens)
        print(f"Initial Tokens: {tokens}")
        print(f"Initial Vocab: {self.vocab}\n")
        for merge_num in range(num_merges):
            pairs = []

            # 2. Counting frequency of adjacent pairs
            for i in range(len(tokens)-1):
                pair = (tokens[i], tokens[i+1])
                pairs.append(pair)
            pair_counts = Counter(pairs)
            if not pair_counts:
                break

            # 3. Finding the most frequent pair
            most_frequent_pair = pair_counts.most_common(1)[0][0]
            self.merges.append(most_frequent_pair)
            merged_token = "".join(most_frequent_pair)

            # 4. Merging the pair in token list
            i = 0
            new_tokens = []
            while i < len(tokens):
                if i < len(tokens)-1 and (tokens[i], tokens[i+1]) == most_frequent_pair:
                    new_tokens.append(merged_token)
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens

            # 5. Updating vocabulary
            self.vocab.add(merged_token)
            # print(f"--- Merge {merge_num} ---")
            # print(f"Pairs: {pairs}")
            # print(f"Pair Counts: {pair_counts}")
            # print(f"Most Frequent Pair: {most_frequent_pair}")
            # print(f"Updated Tokens: {tokens}")
            # print(f"Updated Vocab: {self.vocab}\n")
        return self.vocab

    def tokenize(self, text):
        
        # Starting from character-level tokens including spaces
        tokens = list(text)
        for pair in self.merges:
            merged_token = "".join(pair)
            i = 0
            new_tokens = []
            while i < len(tokens):
                if i < len(tokens)-1 and (tokens[i], tokens[i+1]) == pair:
                    new_tokens.append(merged_token)
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens
        return tokens

text_corpus = 'the dog is a good boy, the cat is a good girl'
bpe = BPE()

# Training with 5 merges
vocab = bpe.train(text_corpus, num_merges=5)
print("Vocabulary after training:", vocab)

# Tokenizing a new sentence
sentence = 'the good dog is a boy'
tokens = bpe.tokenize(sentence)
print("Tokenized sentence:", tokens)

       

Initial Tokens: ['t', 'h', 'e', ' ', 'd', 'o', 'g', ' ', 'i', 's', ' ', 'a', ' ', 'g', 'o', 'o', 'd', ' ', 'b', 'o', 'y', ',', ' ', 't', 'h', 'e', ' ', 'c', 'a', 't', ' ', 'i', 's', ' ', 'a', ' ', 'g', 'o', 'o', 'd', ' ', 'g', 'i', 'r', 'l']
Initial Vocab: {'o', ',', 's', 'g', 'r', 'h', 'l', 'y', ' ', 'a', 'c', 'e', 't', 'b', 'i', 'd'}

Vocabulary after training: {',', 's', 'g', 'r', 'the ', 'l', 'y', 'e', 'd', 'h', ' i', ' ', 'a', 'o', 'c', 'b', ' g', 'th', 'the', 't', 'i'}
Tokenized sentence: ['the', ' g', 'o', 'o', 'd', ' ', 'd', 'o', 'g', ' i', 's', ' ', 'a', ' ', 'b', 'o', 'y']


## Reflective Questions

**1. How does the number of merges affect the size of the final vocabulary?**  
As the number of merges increase, the size of final vocabulary increases. For instance, at the start the initial vocabulary is just the characters. As there are merges happening, the updated vocabulary is individual characters + merged most frequent pairs as tokens.  

**2. How does your tokenizer handle out-of-vocabulary (OOV) words? For example, if the word 'supercalifragilisticexpialidocious' were encountered, how would it be tokenized by your model?**  
In case of BPE the algorithm starts with the individual character level tokens and then perform frequent merges. Hence, no word is fully known. In case, there is some new word in testing phase the algorithm will still apply all the tokens learned during training. e-g. if the model lerned merges like "su", "per", "cal" during training phase and the new word 'supercalifragilisticexpialidocious' appears in the test phase (case of OOV), then the tokenization would look like this (learned tokens + the individual characters of the new word)    
['su', 'per', 'cal', 'i', 'f', 'r', 'a', 'g', 'i', 'l', 'i', 's', 't', 'i', 'c','e','x', 'p','i', 'a', 'l', 'i', 'd', 'o', 'c', 'i', 'o', 'u', 's']


**3. What are the trade-offs between having a very large vocabulary (many merges) and a very small one (few merges)? Consider the impact on model size and the representation of rare words.**  
- In case of a large vocabulary, we have fewer tokens per sentence and common words/sub-words are stored directly. Due to these reasons, we have faster training. This means that such a model will not generalize well to rare words because of less fallback to character level.  
- Small vocabulary (less merges) mean that we have a compact model (smaller embeddings). OOV or rare words will be better handled as they will be broken down into individula level charcters. But this also means that during tokenization, a sentence will be broken down into many tokens (longer sequences) and hence, training will be slower.  

---

## Part 3: The Impact of Corpus Size and Type

### Scenario 3: Training on a Different Corpus

This scenario explores how the characteristics of the training data influence the resulting BPE model.  

#### Problem Description

You are given two different text corpora.

- Corpus A: `'a b c a b c a a b c a a'`
- Corpus B: `'the dog ate the food, the cat ate the mouse'`

Train two separate BPE models, one for each corpus, with `num_merges = 3`. Analyze the resulting vocabularies and the merges learned.  

#### Implementation Steps

1. Use your `train` function from Part 2.
2. Train a BPE model on Corpus A with 3 merges. Print the final vocabulary.
3. Train a BPE model on Corpus B with 3 merges. Print the final vocabulary.
4. Analyze the differences.

#### Implementation Steps

1. Use your `train` function from Part 2.
2. Train a BPE model on Corpus A with 3 merges. Print the final vocabulary.
3. Train a BPE model on Corpus B with 3 merges. Print the final vocabulary.
4. Analyze the differences.

In [ ]:
# Corpus A
corpus_a = 'a b c a b c a a b c a a'
bpe_a = BPE()
print("Training BPE on Corpus A...\n")
vocab_a = bpe_a.train(corpus_a, num_merges=3)
print("Final Vocabulary for Corpus A:", vocab_a)
print("\n" + "-"*50 + "\n")

#  Corpus B
corpus_b = 'the dog ate the food, the cat ate the mouse'
bpe_b = BPE()
print("Training BPE on Corpus B...\n")
vocab_b = bpe_b.train(corpus_b, num_merges=3)
print("Final Vocabulary for Corpus B:", vocab_b)
print("\n" + "-"*50 + "\n")

# Analysis 
print("Analysis of the merges and vocabularies:\n")
print("Corpus A Vocabulary:", vocab_a)
print("Corpus A Merges:", bpe_a.merges)
print()
print("Corpus B Vocabulary:", vocab_b)
print("Corpus B Merges:", bpe_b.merges)


Training BPE on Corpus A...

Initial Tokens: ['a', ' ', 'b', ' ', 'c', ' ', 'a', ' ', 'b', ' ', 'c', ' ', 'a', ' ', 'a', ' ', 'b', ' ', 'c', ' ', 'a', ' ', 'a']
Initial Vocab: {' ', 'b', 'a', 'c'}

Final Vocabulary for Corpus A: {'a b ', ' ', 'a', 'c', 'a ', 'b', 'a b'}

--------------------------------------------------

Training BPE on Corpus B...

Initial Tokens: ['t', 'h', 'e', ' ', 'd', 'o', 'g', ' ', 'a', 't', 'e', ' ', 't', 'h', 'e', ' ', 'f', 'o', 'o', 'd', ',', ' ', 't', 'h', 'e', ' ', 'c', 'a', 't', ' ', 'a', 't', 'e', ' ', 't', 'h', 'e', ' ', 'm', 'o', 'u', 's', 'e']
Initial Vocab: {'o', ',', 's', 'g', 'h', 'f', ' ', 'm', 'a', 'c', 'e', 't', 'u', 'd'}

Final Vocabulary for Corpus B: {'o', ',', 's', 'g', 'th', 'h', 'f', 'e ', 'the ', ' ', 'm', 'a', 'c', 'e', 't', 'u', 'd'}

--------------------------------------------------

Analysis of the merges and vocabularies:

Corpus A Vocabulary: {'a b ', ' ', 'a', 'c', 'a ', 'b', 'a b'}
Corpus A Merges: [('a', ' '), ('a ', 'b'), ('a b

#### Expected Output

The output should show the final vocabulary for both models, clearly labeled.

## Reflective Questions

**1. Compare the vocabularies learned from Corpus A and Corpus B. Why are they different?**  
Corpus A produced abstract subwords (short merged sequences of a,b,c) while Corpus B produced meaningful subwords like th, the which are closer to the real words.   

**2. How does the structure and content of the training data directly affect the subwords that BPE learns?**  
BPE is data-driven. It means that if the training data has repetive short pattern, then there will be abstract tokens. Whereas if the training data is real natural language then the tokens learned will be meaningful subwords and are closer to the real words.  

**3. If you were building a BPE model for a large-scale language model, why would it be crucial to use a diverse and representative training corpus?**  
If the training corpus is diverse and representative across various disciplines, model will learn balanced vocabulary of subwords that will be generalize across sub-words. On the other hand, if the training corpus is not diverse then model will overfit to frequent patterns in that discipline only.


---

## Part 4: Applying BPE to a New Language

### Scenario 4: Training BPE on an Urdu corpus

This final part extends the BPE implementation to a different language, Urdu, to observe how the algorithm adapts to a new script and linguistic structure.  

#### Problem Description

You are given a corpus of Urdu text in the file `datasets/urdu_corpus.txt`. Your task is to train your BPE model on this text, analyze the resulting subwords, and tokenize a new sentence.  

#### Implementation Steps

1. Print the final vocabulary, including the base characters and the learned merges (use num_merges = 10).
2. Tokenize the given test sentence and print the resultant tokens

In [ ]:
from collections import Counter
class BPE:
    def __init__(self):
        self.vocab = set()
        self.merges = []

    def train(self, text_corpus, num_merges):
        # Character-level tokens excluding spaces
        tokens = [ch for ch in text_corpus if ch != ' ']
        self.vocab = set(tokens)
        print(f"Initial Tokens: {tokens[:50]}...")  # showing first 50
        print(f"Initial Vocab: {self.vocab}\n")
        for merge_num in range(1, num_merges+1):
            pairs = [(tokens[i], tokens[i+1]) for i in range(len(tokens)-1)]
            pair_counts = Counter(pairs)
            if not pair_counts:
                break

            # Most frequent pair
            most_frequent_pair = pair_counts.most_common(1)[0][0]
            self.merges.append(most_frequent_pair)
            merged_token = "".join(most_frequent_pair)

            # Merging tokens
            i = 0
            new_tokens = []
            while i < len(tokens):
                if i < len(tokens)-1 and (tokens[i], tokens[i+1]) == most_frequent_pair:
                    new_tokens.append(merged_token)
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens

            # Updating vocab
            self.vocab.add(merged_token)
        return self.vocab

    def tokenize(self, text):
        tokens = [ch for ch in text if ch != ' ']
        for pair in self.merges:
            merged_token = "".join(pair)
            i = 0
            new_tokens = []
            while i < len(tokens):
                if i < len(tokens)-1 and (tokens[i], tokens[i+1]) == pair:
                    new_tokens.append(merged_token)
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens
        return tokens

# on urdu_corpus.txt
# 1. Loading Urdu corpus
with open("datasets/urdu_corpus.txt", "r", encoding="utf-8") as f:
    urdu_corpus = f.read()
bpe_urdu = BPE()

# 2. Training with 10 merges
print("Training BPE on Urdu corpus...\n")
vocab = bpe_urdu.train(urdu_corpus, num_merges=10)
print("\nFinal Vocabulary:", vocab)

# 3. Tokenizing new test sentence
test_sentence = """
ٹچ پیڈ یوایس بھی کی پیڈ ہے، یعنی اسے یو ایس بی کیبل کے ذریعے کمپیوٹر سے منسلک کیا جاسکتا ہے۔
"""
tokens = bpe_urdu.tokenize(test_sentence)
print("\nTokenized Sentence:")
print(tokens)


Training BPE on Urdu corpus...

Initial Tokens: ['ی', 'و', 'ٹ', 'ی', 'و', 'ب', 'ن', 'ے', '8', '0', 'ل', 'ا', 'ک', 'ھ', 'س', 'ے', 'ز', 'ا', 'ئ', 'د', 'خ', 'ل', 'ا', 'ف', 'ض', 'ا', 'ب', 'ط', 'ہ', 'و', 'ی', 'ڈ', 'ی', 'و', 'ز', 'ض', 'ا', 'ئ', 'ع', 'ک', 'ر', 'د', 'ی', 'ں', '\n', 'ی', 'و', 'ر', 'ی', 'ن']...
Initial Vocab: {'ف', 'R', 'r', 'N', 'l', '2', 'D', '9', 'ُ', 'e', 'د', 'V', 'ذ', '‘', 'P', 'ج', '5', 'a', 'ژ', 'ٔ', 'u', 'ح', '!', '6', 'ط', 'ڑ', '’', 'b', 'خ', 'H', 'ا', 'ِ', 'w', 't', ')', '\xa0', '\n', ',', 'ٓ', '0', 's', 'َ', 'ن', 'غ', 'ؤ', '3', 'd', 'ر', 'I', '1', 'h', 'چ', 'ء', 'T', 'ص', "'", 'ز', 'o', 'ک', '-', 'ت', 'ظ', 'ٰ', 'ب', '…', 'ں', 'و', 'L', ':', 'ئ', 'g', 'y', 'آ', '8', 'ے', 'م', 'س', '،', '؛', 'j', 'Z', '"', 'ھ', 'k', 'ْ', 'ل', 'ع', 'ہ', 'p', 'M', 'ض', 'B', 'O', 'گ', 'پ', 'ق', '(', 'K', 'm', '7', 'ش', 'ّ', 'ی', 'C', 'ث', 'S', 'A', 'ك', 'ٹ', '۔', '؟', 'c', '.', 'n', 'ي', 'F', '4', 'ڈ', 'i'}


Final Vocabulary: {'ف', 'R', 'r', 'N', 'l', '2', 'D', '9', 'ُ', 'e', 'ار', 'د', 

## Reflective Question

BPE is called a language-agnostic algorithm; an algorithm does not rely on any pre-existing linguistic knowledge of a language.

Comment on the features of the algorithm that make it adaptive to different languages.

**Answer:**  
BPE does not “know” about words, grammar, or morphology. Instead, it relies on statistics over character sequences. This makes it agnostic to English, Urdu, Chinese, or any other language.  
Following are the features of the algorithm that makes it adaptive to different languages:  
- BPE always starts with characters and since there are characters in every language, it has a universal starting point.
- BPE merges adjacent pairs based purely on frequency and not on grammar. There are no linguistic dependent hard core rules and everything is data driven. Hence, it naturally captures most frequent words/sub words across different languages.
- BPE is very robust to new or OOV words. If a word isn't in the vocab then BPE falls back to smaller units (characters).
